In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob, os
import shutil

In [3]:
df = pd.read_pickle('timex_inventory.pkl')
df.sample(5)

,dts,dto,fnamec1,datenum,doy,tide,hsn,az,zen,avg1,...,red1,green1,blue1,red2,green2,blue2,sharp1,sharp2,contrast1,contrast2
492,2021-03-29T15:00:00,2021-03-29 15:00:00,1617030000.c1.timex.jpg,738244.6250,88.0,0.9781,-2.0718,134.2378,48.0033,118.9990,...,152.7925,117.2956,86.9089,123.6400,128.2855,134.7291,0.0000,0.0,0.0000,0.0
242,2021-03-19T20:00:00,2021-03-19 20:00:00,1616184000.c1.timex.jpg,738234.8333,78.0,0.6844,2.9282,233.2150,56.7021,115.4437,...,122.7289,115.3023,108.2998,128.6437,121.4568,117.0920,0.0000,0.0,0.0000,0.0
1484,2021-05-22T21:30:00,2021-05-22 21:30:00,1621719000.c1.timex.jpg,738298.8958,142.0,-0.2507,4.4282,271.2050,59.7242,109.3569,...,123.9411,109.8627,94.2670,127.7640,111.4977,94.2676,0.0000,0.0,0.0000,0.0
1463,2021-05-22T11:00:00,2021-05-22 11:00:00,1621681200.c1.timex.jpg,738298.4583,142.0,0.8254,-6.0718,74.3469,76.6318,114.0190,...,127.4600,114.6123,99.9846,131.4746,119.7706,108.5953,0.0001,0.0,0.0114,0.0
234,2021-03-19T16:00:00,2021-03-19 16:00:00,1616169600.c1.timex.jpg,738234.6667,78.0,-0.7528,-1.0718,154.2645,45.3464,103.7315,...,108.0721,103.8150,99.3072,127.6434,118.1181,107.5828,0.0000,0.0,0.0000,0.0


In [4]:
# remove dark images
df = df.drop(df[df['avg1']< 80].index)
df = df.drop(df[df['avg2']< 80].index)

In [5]:
dfs = df.sample(20)

In [6]:
# move the c2 images to a new directory
old_dir = 'F:/timex/'
new_dir = 'F:/timex_movie_input/'
mov_dir = 'F:/timex_movie/'
if not os.path.exists(new_dir):
    os.makedirs(new_dir)
if not os.path.exists(mov_dir):
    os.makedirs(mov_dir)

ic = 0
for index, row in df.iterrows():
    # old_file = row['fnamec1'].replace('c1','c2')
    old_file = row['fnamec1']
    old_path = old_dir+old_file
    # new_name = 'c2{:04.0f}.jpg'.format(ic)
    new_name = 'c1{:04.0f}.jpg'.format(ic)
    ic = ic+1
    new_path = new_dir+new_name
    print(old_path, new_path)
    shutil.copy(old_path, new_path)

F:/timex/1614895200.c1.timex.jpg F:/timex_movie_input/c10000.jpg
F:/timex/1614967200.c1.timex.jpg F:/timex_movie_input/c10001.jpg
F:/timex/1615152600.c1.timex.jpg F:/timex_movie_input/c10002.jpg
F:/timex/1615213800.c1.timex.jpg F:/timex_movie_input/c10003.jpg
F:/timex/1615215600.c1.timex.jpg F:/timex_movie_input/c10004.jpg
F:/timex/1615219200.c1.timex.jpg F:/timex_movie_input/c10005.jpg
F:/timex/1615222800.c1.timex.jpg F:/timex_movie_input/c10006.jpg
F:/timex/1615224600.c1.timex.jpg F:/timex_movie_input/c10007.jpg
F:/timex/1615226400.c1.timex.jpg F:/timex_movie_input/c10008.jpg
F:/timex/1615228200.c1.timex.jpg F:/timex_movie_input/c10009.jpg
F:/timex/1615230000.c1.timex.jpg F:/timex_movie_input/c10010.jpg
F:/timex/1615231800.c1.timex.jpg F:/timex_movie_input/c10011.jpg
F:/timex/1615233600.c1.timex.jpg F:/timex_movie_input/c10012.jpg
F:/timex/1615235400.c1.timex.jpg F:/timex_movie_input/c10013.jpg
F:/timex/1615237200.c1.timex.jpg F:/timex_movie_input/c10014.jpg
F:/timex/1615291200.c1.ti

F:/timex/1616326200.c1.timex.jpg F:/timex_movie_input/c10240.jpg
F:/timex/1616328000.c1.timex.jpg F:/timex_movie_input/c10241.jpg
F:/timex/1616329800.c1.timex.jpg F:/timex_movie_input/c10242.jpg
F:/timex/1616331600.c1.timex.jpg F:/timex_movie_input/c10243.jpg
F:/timex/1616333400.c1.timex.jpg F:/timex_movie_input/c10244.jpg
F:/timex/1616335200.c1.timex.jpg F:/timex_movie_input/c10245.jpg
F:/timex/1616337000.c1.timex.jpg F:/timex_movie_input/c10246.jpg
F:/timex/1616338800.c1.timex.jpg F:/timex_movie_input/c10247.jpg
F:/timex/1616340600.c1.timex.jpg F:/timex_movie_input/c10248.jpg
F:/timex/1616342400.c1.timex.jpg F:/timex_movie_input/c10249.jpg
F:/timex/1616344200.c1.timex.jpg F:/timex_movie_input/c10250.jpg
F:/timex/1616346000.c1.timex.jpg F:/timex_movie_input/c10251.jpg
F:/timex/1616347800.c1.timex.jpg F:/timex_movie_input/c10252.jpg
F:/timex/1616349600.c1.timex.jpg F:/timex_movie_input/c10253.jpg
F:/timex/1616351400.c1.timex.jpg F:/timex_movie_input/c10254.jpg
F:/timex/1616353200.c1.ti

F:/timex/1616779800.c1.timex.jpg F:/timex_movie_input/c10371.jpg
F:/timex/1616781600.c1.timex.jpg F:/timex_movie_input/c10372.jpg
F:/timex/1616783400.c1.timex.jpg F:/timex_movie_input/c10373.jpg
F:/timex/1616785200.c1.timex.jpg F:/timex_movie_input/c10374.jpg
F:/timex/1616787000.c1.timex.jpg F:/timex_movie_input/c10375.jpg
F:/timex/1616788800.c1.timex.jpg F:/timex_movie_input/c10376.jpg
F:/timex/1616790600.c1.timex.jpg F:/timex_movie_input/c10377.jpg
F:/timex/1616792400.c1.timex.jpg F:/timex_movie_input/c10378.jpg
F:/timex/1616794200.c1.timex.jpg F:/timex_movie_input/c10379.jpg
F:/timex/1616796000.c1.timex.jpg F:/timex_movie_input/c10380.jpg
F:/timex/1616797800.c1.timex.jpg F:/timex_movie_input/c10381.jpg
F:/timex/1616842800.c1.timex.jpg F:/timex_movie_input/c10382.jpg
F:/timex/1616844600.c1.timex.jpg F:/timex_movie_input/c10383.jpg
F:/timex/1616846400.c1.timex.jpg F:/timex_movie_input/c10384.jpg
F:/timex/1616848200.c1.timex.jpg F:/timex_movie_input/c10385.jpg
F:/timex/1616850000.c1.ti

F:/timex/1618506000.c1.timex.jpg F:/timex_movie_input/c10622.jpg
F:/timex/1618507800.c1.timex.jpg F:/timex_movie_input/c10623.jpg
F:/timex/1618509600.c1.timex.jpg F:/timex_movie_input/c10624.jpg
F:/timex/1618511400.c1.timex.jpg F:/timex_movie_input/c10625.jpg
F:/timex/1618513200.c1.timex.jpg F:/timex_movie_input/c10626.jpg
F:/timex/1618515000.c1.timex.jpg F:/timex_movie_input/c10627.jpg
F:/timex/1618516800.c1.timex.jpg F:/timex_movie_input/c10628.jpg
F:/timex/1618518600.c1.timex.jpg F:/timex_movie_input/c10629.jpg
F:/timex/1618520400.c1.timex.jpg F:/timex_movie_input/c10630.jpg
F:/timex/1618522200.c1.timex.jpg F:/timex_movie_input/c10631.jpg
F:/timex/1618524000.c1.timex.jpg F:/timex_movie_input/c10632.jpg
F:/timex/1618525800.c1.timex.jpg F:/timex_movie_input/c10633.jpg
F:/timex/1618572600.c1.timex.jpg F:/timex_movie_input/c10634.jpg
F:/timex/1618574400.c1.timex.jpg F:/timex_movie_input/c10635.jpg
F:/timex/1618576200.c1.timex.jpg F:/timex_movie_input/c10636.jpg
F:/timex/1618578000.c1.ti

F:/timex/1618943400.c1.timex.jpg F:/timex_movie_input/c10754.jpg
F:/timex/1618945200.c1.timex.jpg F:/timex_movie_input/c10755.jpg
F:/timex/1618947000.c1.timex.jpg F:/timex_movie_input/c10756.jpg
F:/timex/1618948800.c1.timex.jpg F:/timex_movie_input/c10757.jpg
F:/timex/1618950600.c1.timex.jpg F:/timex_movie_input/c10758.jpg
F:/timex/1618952400.c1.timex.jpg F:/timex_movie_input/c10759.jpg
F:/timex/1618954200.c1.timex.jpg F:/timex_movie_input/c10760.jpg
F:/timex/1618956000.c1.timex.jpg F:/timex_movie_input/c10761.jpg
F:/timex/1618957800.c1.timex.jpg F:/timex_movie_input/c10762.jpg
F:/timex/1618959600.c1.timex.jpg F:/timex_movie_input/c10763.jpg
F:/timex/1618999200.c1.timex.jpg F:/timex_movie_input/c10764.jpg
F:/timex/1619001000.c1.timex.jpg F:/timex_movie_input/c10765.jpg
F:/timex/1619002800.c1.timex.jpg F:/timex_movie_input/c10766.jpg
F:/timex/1619004600.c1.timex.jpg F:/timex_movie_input/c10767.jpg
F:/timex/1619006400.c1.timex.jpg F:/timex_movie_input/c10768.jpg
F:/timex/1619008200.c1.ti

F:/timex/1619710200.c1.timex.jpg F:/timex_movie_input/c10985.jpg
F:/timex/1619712000.c1.timex.jpg F:/timex_movie_input/c10986.jpg
F:/timex/1619713800.c1.timex.jpg F:/timex_movie_input/c10987.jpg
F:/timex/1619715600.c1.timex.jpg F:/timex_movie_input/c10988.jpg
F:/timex/1619717400.c1.timex.jpg F:/timex_movie_input/c10989.jpg
F:/timex/1619719200.c1.timex.jpg F:/timex_movie_input/c10990.jpg
F:/timex/1619721000.c1.timex.jpg F:/timex_movie_input/c10991.jpg
F:/timex/1619722800.c1.timex.jpg F:/timex_movie_input/c10992.jpg
F:/timex/1619724600.c1.timex.jpg F:/timex_movie_input/c10993.jpg
F:/timex/1619726400.c1.timex.jpg F:/timex_movie_input/c10994.jpg
F:/timex/1619728200.c1.timex.jpg F:/timex_movie_input/c10995.jpg
F:/timex/1619730000.c1.timex.jpg F:/timex_movie_input/c10996.jpg
F:/timex/1619731800.c1.timex.jpg F:/timex_movie_input/c10997.jpg
F:/timex/1619733600.c1.timex.jpg F:/timex_movie_input/c10998.jpg
F:/timex/1619735400.c1.timex.jpg F:/timex_movie_input/c10999.jpg
F:/timex/1619780400.c1.ti

F:/timex/1621204200.c1.timex.jpg F:/timex_movie_input/c11217.jpg
F:/timex/1621206000.c1.timex.jpg F:/timex_movie_input/c11218.jpg
F:/timex/1621207800.c1.timex.jpg F:/timex_movie_input/c11219.jpg
F:/timex/1621245600.c1.timex.jpg F:/timex_movie_input/c11220.jpg
F:/timex/1621247400.c1.timex.jpg F:/timex_movie_input/c11221.jpg
F:/timex/1621249200.c1.timex.jpg F:/timex_movie_input/c11222.jpg
F:/timex/1621251000.c1.timex.jpg F:/timex_movie_input/c11223.jpg
F:/timex/1621252800.c1.timex.jpg F:/timex_movie_input/c11224.jpg
F:/timex/1621254600.c1.timex.jpg F:/timex_movie_input/c11225.jpg
F:/timex/1621256400.c1.timex.jpg F:/timex_movie_input/c11226.jpg
F:/timex/1621258200.c1.timex.jpg F:/timex_movie_input/c11227.jpg
F:/timex/1621260000.c1.timex.jpg F:/timex_movie_input/c11228.jpg
F:/timex/1621261800.c1.timex.jpg F:/timex_movie_input/c11229.jpg
F:/timex/1621263600.c1.timex.jpg F:/timex_movie_input/c11230.jpg
F:/timex/1621265400.c1.timex.jpg F:/timex_movie_input/c11231.jpg
F:/timex/1621267200.c1.ti

F:/timex/1621960200.c1.timex.jpg F:/timex_movie_input/c11457.jpg
F:/timex/1621962000.c1.timex.jpg F:/timex_movie_input/c11458.jpg
F:/timex/1621963800.c1.timex.jpg F:/timex_movie_input/c11459.jpg
F:/timex/1621965600.c1.timex.jpg F:/timex_movie_input/c11460.jpg
F:/timex/1621967400.c1.timex.jpg F:/timex_movie_input/c11461.jpg
F:/timex/1621969200.c1.timex.jpg F:/timex_movie_input/c11462.jpg
F:/timex/1621971000.c1.timex.jpg F:/timex_movie_input/c11463.jpg
F:/timex/1621972800.c1.timex.jpg F:/timex_movie_input/c11464.jpg
F:/timex/1621974600.c1.timex.jpg F:/timex_movie_input/c11465.jpg
F:/timex/1621976400.c1.timex.jpg F:/timex_movie_input/c11466.jpg
F:/timex/1621978200.c1.timex.jpg F:/timex_movie_input/c11467.jpg
F:/timex/1621980000.c1.timex.jpg F:/timex_movie_input/c11468.jpg
F:/timex/1621981800.c1.timex.jpg F:/timex_movie_input/c11469.jpg
F:/timex/1621983600.c1.timex.jpg F:/timex_movie_input/c11470.jpg
F:/timex/1621985400.c1.timex.jpg F:/timex_movie_input/c11471.jpg
F:/timex/1622023200.c1.ti

F:/timex/1622399400.c1.timex.jpg F:/timex_movie_input/c11597.jpg
F:/timex/1622401200.c1.timex.jpg F:/timex_movie_input/c11598.jpg
F:/timex/1622403000.c1.timex.jpg F:/timex_movie_input/c11599.jpg
F:/timex/1622404800.c1.timex.jpg F:/timex_movie_input/c11600.jpg
F:/timex/1622406600.c1.timex.jpg F:/timex_movie_input/c11601.jpg
F:/timex/1622408400.c1.timex.jpg F:/timex_movie_input/c11602.jpg
F:/timex/1622410200.c1.timex.jpg F:/timex_movie_input/c11603.jpg
F:/timex/1622412000.c1.timex.jpg F:/timex_movie_input/c11604.jpg
F:/timex/1622413800.c1.timex.jpg F:/timex_movie_input/c11605.jpg
F:/timex/1622415600.c1.timex.jpg F:/timex_movie_input/c11606.jpg
F:/timex/1622455200.c1.timex.jpg F:/timex_movie_input/c11607.jpg
F:/timex/1622457000.c1.timex.jpg F:/timex_movie_input/c11608.jpg
F:/timex/1622458800.c1.timex.jpg F:/timex_movie_input/c11609.jpg
F:/timex/1622460600.c1.timex.jpg F:/timex_movie_input/c11610.jpg
F:/timex/1622462400.c1.timex.jpg F:/timex_movie_input/c11611.jpg
F:/timex/1622464200.c1.ti

In [9]:
# I used this...
ffmpeg_cmd = 'ffmpeg -framerate 10 -i F:/timex_movie_input/c2%04d.jpg  -q:v 3 F/timex_movie/timex.avi'
#print(ffmpeg_cmd)
# Dan uses these switches. He also uses glob, but that option is not in the conda dist of ffmpeg
ffmpeg_cmd = "ffmpeg -framerate 10 -i F:/timex_movie_input/c1%04d.jpg -c:v libx264 -pix_fmt yuv420p -vf scale=-1:1024 F:/timex_movie/c1timex2.mp4"
print(ffmpeg_cmd)
if os.system(ffmpeg_cmd) == 0:
    print('Success')


ffmpeg -framerate 10 -i F:/timex_movie_input/c1%04d.jpg -c:v libx264 -pix_fmt yuv420p -vf scale=-1:1024 F:/timex_movie/c1timex2.mp4
Success
